In [1]:
import pandas as pd
import numpy as np

In [2]:
train=np.array(pd.read_csv("w2v_train.csv")).tolist()
test=np.array(pd.read_csv("w2v_test.csv")).tolist()

In [3]:
target=pd.read_csv("train_all.csv")

In [4]:
age_target=np.array(target["age"])
gender_target=np.array(target["gender"])-1

In [5]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 721 kB/s eta 0:00:01


In [12]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(train,gender_target,test_size=0.04)
 
 
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss', 'auc'},
    'num_leaves': 5,
    'max_depth': 6,
    'min_data_in_leaf': 450,
    'learning_rate': 0.3,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5,
    'lambda_l1': 1,  
    'lambda_l2': 0.001,  # 越小l2正则程度越高
    'min_gain_to_split': 0.2,
    'verbose': 5,
    'is_unbalance': True
}
 
# train
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
 
print('Start predicting...')
 
preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
# 导出结果
threshold = 0.5
gender_pred=[]
for pred in preds:
    result = 1 if pred > threshold else 0
    gender_pred.append(result)
# 导出特征重要性
# importance = gbm.feature_importance()
# names = gbm.feature_name()
# with open('./feature_importance.txt', 'w+') as file:
#     for index, im in enumerate(importance):
#         string = names[index] + ', ' + str(im) + '\n'
#         file.write(string)

Start training...
[1]	valid_0's auc: 0.829458	valid_0's binary_logloss: 0.548668
Training until validation scores don't improve for 10 rounds
[2]	valid_0's auc: 0.886367	valid_0's binary_logloss: 0.50501
[3]	valid_0's auc: 0.900051	valid_0's binary_logloss: 0.476653
[4]	valid_0's auc: 0.911446	valid_0's binary_logloss: 0.4465
[5]	valid_0's auc: 0.922418	valid_0's binary_logloss: 0.422205
[6]	valid_0's auc: 0.926093	valid_0's binary_logloss: 0.400822
[7]	valid_0's auc: 0.932085	valid_0's binary_logloss: 0.382952
[8]	valid_0's auc: 0.935648	valid_0's binary_logloss: 0.368472
[9]	valid_0's auc: 0.937514	valid_0's binary_logloss: 0.359334
[10]	valid_0's auc: 0.938903	valid_0's binary_logloss: 0.349532
[11]	valid_0's auc: 0.939962	valid_0's binary_logloss: 0.340495
[12]	valid_0's auc: 0.942	valid_0's binary_logloss: 0.333119
[13]	valid_0's auc: 0.943463	valid_0's binary_logloss: 0.325677
[14]	valid_0's auc: 0.944679	valid_0's binary_logloss: 0.319374
[15]	valid_0's auc: 0.946004	valid_0's b

In [13]:
t=0
for i in range(0,len(y_test)):
    if gender_pred[i]==y_test[i]:
        t+=1
acct=t/len(gender_pred)
print(acct)#gender模型准确率

0.9128888888888889


In [14]:
print('Start predicting...')
 
preds = gbm.predict(test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
# 导出结果
threshold = 0.5
gender_pred=[]
for pred in preds:
    result = 2 if pred > threshold else 1
    gender_pred.append(result)#预测结果

Start predicting...


In [15]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
 
X_train,X_test,y_train,y_test=train_test_split(train,age_target,test_size=0.04)
 
 
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 11,
    'metric': 'multi_error',
    'num_leaves': 300,
    'min_data_in_leaf': 100,
    'learning_rate': 0.2,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'lambda_l1': 0.4,
    'lambda_l2': 0.5,
    'min_gain_to_split': 0.2,
    'verbose': 5,
    'is_unbalance': True
}
 
# train
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
 
print('Start predicting...')
 
preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
# 导出结果
age_pred=[]
for pred in preds:
    result = prediction = int(np.argmax(pred))
    age_pred.append(result)
# 导出特征重要性
# importance = gbm.feature_importance()
# names = gbm.feature_name()
# with open('./feature_importance.txt', 'w+') as file:
#     for index, im in enumerate(importance):
#         string = names[index] + ', ' + str(im) + '\n'
#         file.write(string)

Start training...
[1]	valid_0's multi_error: 0.77
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_error: 0.752861
[3]	valid_0's multi_error: 0.732
[4]	valid_0's multi_error: 0.712722
[5]	valid_0's multi_error: 0.69725
[6]	valid_0's multi_error: 0.688333
[7]	valid_0's multi_error: 0.682111
[8]	valid_0's multi_error: 0.678194
[9]	valid_0's multi_error: 0.674056
[10]	valid_0's multi_error: 0.670833
[11]	valid_0's multi_error: 0.668917
[12]	valid_0's multi_error: 0.665833
[13]	valid_0's multi_error: 0.663139
[14]	valid_0's multi_error: 0.662417
[15]	valid_0's multi_error: 0.662278
[16]	valid_0's multi_error: 0.661444
[17]	valid_0's multi_error: 0.661611
[18]	valid_0's multi_error: 0.661694
[19]	valid_0's multi_error: 0.659028
[20]	valid_0's multi_error: 0.658611
[21]	valid_0's multi_error: 0.657917
[22]	valid_0's multi_error: 0.65675
[23]	valid_0's multi_error: 0.654694
[24]	valid_0's multi_error: 0.654583
[25]	valid_0's multi_error: 0.653833
[26]	valid_0's

In [16]:
t=0
for i in range(0,len(y_test)):
    if age_pred[i]==y_test[i]:
        t+=1
acct=t/len(age_pred)
print(acct)#age模型准确率

0.3586944444444444


In [17]:
print('Start predicting...')
 
preds = gbm.predict(test, num_iteration=gbm.best_iteration)  # 输出的是概率结果
 
 
# 导出结果
age_pred=[]
for pred in preds:
    result = prediction = int(np.argmax(pred))
    age_pred.append(result)

Start predicting...


In [18]:
submission=pd.DataFrame(columns=("user_id","predicted_age","predicted_gender"))

In [19]:
test_all=pd.read_csv("test_all.csv")

In [20]:
submission["user_id"]=test_all["user_id"]
submission["predicted_age"]=age_pred
submission["predicted_gender"]=gender_pred

In [21]:
submission.to_csv("submission_5 30_2(new model).csv",index=False)

In [22]:
submission

,user_id,predicted_age,predicted_gender
0,3000001,2,1
1,3000002,5,2
2,3000003,2,2
3,3000004,2,1
4,3000005,3,1
...,...,...,...
999995,3999996,3,1
999996,3999997,2,1
999997,3999998,2,1
999998,3999999,2,1
